In [1]:
import tensorflow as tf
import ktrain

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

Default GPU Device:/device:GPU:0


In [2]:

import ktrain
from ktrain import text

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

num_folds = 5

run_precision = []
run_recall = []
run_f1score = []
run_accuracy = []

kf = KFold(n_splits=5, shuffle=False, random_state=125)

In [4]:
data = pd.read_csv('sstubsIn_edited.csv')

In [5]:
print(data.shape)

(10231, 5)


In [6]:
#print("pre-processing train data...")

from sklearn.utils import shuffle
from tqdm import tqdm

#label_names = ["bugType"]
#y_train = data[label_names].values

raw_docs_train = data['sourceBeforeFix'].tolist()
#raw_docs_test = test_data['message'].tolist() 
num_classes = 4

#processed_docs_train = []
#for doc in tqdm(raw_docs_train):
#    tokens = word_tokenize(doc)
#    filtered = [word for word in tokens if word not in stop_words]
#    processed_docs_train.append(" ".join(filtered))
#end for
'''
processed_docs_test = []
for doc in tqdm(raw_docs_test):
    tokens = word_tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(filtered))
#end for
'''


#print("tokenizing input data...")
#tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
#tokenizer.fit_on_texts(processed_docs_train )  #leaky
#word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
#word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
#word_index = tokenizer.word_index
#print("dictionary size: ", len(word_index))

#pad sequences
#word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_train = raw_docs_train
word_seq_train = np.array(word_seq_train)

#data.bugType.value_counts()

In [7]:
class_names = [0,1,2,3]

In [8]:
encoding = {
    'CHANGE_IDENTIFIER': 0, 
    'CHANGE_MODIFIER': 0,
    'CHANGE_NUMERAL': 0, 
    'SWAP_BOOLEAN_LITERAL': 0,
     'DIFFERENT_METHOD_SAME_ARGS': 1, 
     'OVERLOAD_METHOD_MORE_ARGS': 1, 
    'OVERLOAD_METHOD_DELETED_ARGS': 1,
     'CHANGE_CALLER_IN_FUNCTION_CALL': 1, 
     'SWAP_ARGUMENTS': 1,
     'CHANGE_OPERATOR': 2, 
    'CHANGE_UNARY_OPERATOR': 2, 
    'CHANGE_OPERAND': 2, 
     'LESS_SPECIFIC_IF': 3,
    'MORE_SPECIFIC_IF': 3,
     'ADD_THROWS_EXCEPTION': 3, 
      'DELETE_THROWS_EXCEPTION':3,
       
}

In [9]:
y_train_initial= data['bugType'].values


y_train_initial = [encoding[x] for x in y_train_initial]
y_train_initial=np.array(y_train_initial)

In [10]:
import tensorflow as tf
from sklearn import metrics
from sklearn.model_selection import train_test_split
count = 1

for train_index, test_index in kf.split(word_seq_train):
    x_trn, x_tst = word_seq_train[train_index], word_seq_train[test_index]
    y_trn, y_tst = y_train_initial[train_index], y_train_initial[test_index]
    
    x_new_train, x_val, y_new_train, y_val= train_test_split(x_trn, y_trn, test_size=0.11115, random_state=125)
    
    print("\nFold ", count)
    
    (x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_new_train, y_train=y_new_train,
                                                                      x_test=x_tst, y_test=y_tst,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=150, 
                                                                       max_features=1000)
    
    model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
    
    learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=10)
    
    
    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
    
    learner.fit_onecycle(2e-5, 10, callbacks=[callbacks])
    
    predictor = ktrain.get_predictor(learner.model, preproc)
    predictor.get_classes()
    
    (x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_val, y_train=y_val,
                                                                       x_test=x_val, y_test=y_val,
                                                                       
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=150, 
                                                                       max_features=1000)
    
    learner.validate(val_data=(x_test, y_test), class_names=class_names)
    
    '''
    print(metrics.classification_report(y_tst, y_pred))
    
    precision = precision_score(y_tst, y_pred, pos_label=1)
    recall = recall_score(y_tst, y_pred, pos_label=1)
    f1score = f1_score(y_tst, y_pred, pos_label=1)
    lstm_accuracy = accuracy_score(y_tst, y_pred)

    run_accuracy.append(accuracy)
    run_f1score.append(f1score)
    run_precision.append(precision)
    run_recall.append(recall)
    '''
    count = count+1
    #X_train = data['sourceBeforeFix'].tolist()


Fold  1
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification
Is Multi-Label? False
maxlen is 150
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
728/728 [==============================] - 219s 285ms/step - loss: 0.9252 - accuracy: 0.5700 - val_loss: 0.5760 - val_accuracy: 0.7103
Epoch 2/10
728/728 [==============================] - 212s 291ms/step - loss: 0.6025 - accuracy: 0.6950 - val_loss: 0.5226 - val_accuracy: 0.7015
Epoch 3/10
728/728 [==============================] - 209s 287ms/step - loss: 0.5313 - accuracy: 0.7181 - val_loss: 0.5210 - val_accuracy: 0.7518
Epoch 4/10
728/728 [==============================] - 203s 279ms/step - loss: 0.4978 - accuracy: 0.7320 - val_loss: 0.5144 - val_accuracy: 0.7518
Epoch 5/10
728/728 [==============================] - 210s 288ms/step - loss: 0.4640 - accuracy: 0.7458 - val_loss: 0.5461 - val_accuracy: 0.6947
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification
              precision    recall  f1-score   support

           0       0.72      0.89      0.79       563
           1       0.59      0.34      0.43       244
           2       0.76      0.53      0.62        53
           3       0.69      0.48      0.56        50

    accuracy                           0.70       910
   macro avg       0.69      0.56      0.60       910
weighted avg       0.68      0.70      0.67       910


Fold  2
preprocessing train...
language: en


...

task: text classification
              precision    recall  f1-score   support

           0       0.76      0.91      0.83       587
           1       0.70      0.40      0.51       237
           2       0.51      0.56      0.53        36
           3       0.69      0.48      0.56        50

    accuracy                           0.74       910
   macro avg       0.66      0.59      0.61       910
weighted avg       0.73      0.74      0.72       910


Fold  5
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification
Is Multi-Label? False
maxlen is 150
done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
728/728 [==============================] - 212s 276ms/step - loss: 0.9990 - accuracy: 0.5600 - val_loss: 0.7149 - val_accuracy: 0.6290
Epoch 2/10
728/728 [==============================] - 198s 272ms/step - loss: 0.5885 - accuracy: 0.6997 - val_loss: 0.8347 - val_accuracy: 0.6447
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification
              precision    recall  f1-score   support

           0       0.96      0.63      0.76       599
           1       0.50      0.97      0.66       221
           2       0.52      0.62      0.56        53
           3       0.57      0.35      0.43        37

    accuracy                           0.70       910
   macro avg       0.63      0.64      0.60       910
weighted avg       0.80      0.70      0.71       910



In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train_new, y_test= train_test_split(X_train, y_train, test_size=0.1, random_state=125)

In [13]:
x_train, x_val, y_train, y_val= train_test_split(x_train,  y_train_new, test_size=0.21, random_state=125)


In [14]:
(x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=3500)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [15]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)


Is Multi-Label? False
maxlen is 350
done.


In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 350)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 350)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 350, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 350, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [17]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=16)

/home/amiangshu/anaconda3/envs/dsaktrain/lib/python3.8/site-packages/ktrain/__init__.py:90: UserWarning: For a GPU with 12GB of RAM, the following maxima apply:
        sequence len=64, max_batch_size=64
        sequence len=128, max_batch_size=32
        sequence len=256, max_batch_size=16
        sequence len=320, max_batch_size=14
        sequence len=384, max_batch_size=12
        sequence len=512, max_batch_size=6
        
        You've exceeded these limits.
        If using a GPU with <=12GB of memory, you may run out of memory during training.
        If necessary, adjust sequence length or batch size based on above.
  if wrn: I.warnings.warn(msg)


In [18]:
import tensorflow as tf
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)

In [19]:
learner.fit_onecycle(2e-5, 10, callbacks=[callbacks])




begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
455/455 [==============================] - 323s 686ms/step - loss: 1.0258 - accuracy: 0.5128 - val_loss: 0.6159 - val_accuracy: 0.6758
Epoch 2/10
455/455 [==============================] - 312s 685ms/step - loss: 0.5848 - accuracy: 0.7065 - val_loss: 0.5852 - val_accuracy: 0.6953
Epoch 3/10
455/455 [==============================] - 311s 684ms/step - loss: 0.5399 - accuracy: 0.7209 - val_loss: 0.5709 - val_accuracy: 0.6973
Epoch 4/10
455/455 [==============================] - 306s 672ms/step - loss: 0.4895 - accuracy: 0.7481 - val_loss: 0.6039 - val_accuracy: 0.6787


In [20]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

[0, 1, 2, 3]

In [21]:
(x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_val, y_train=y_val,
                                                                       x_test=x_val, y_test=y_val,
                                                                       
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=3500)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [22]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)


              precision    recall  f1-score   support

           0       0.76      0.78      0.77      1183
           1       0.57      0.59      0.58       549
           2       0.60      0.50      0.55       101
           3       0.62      0.43      0.51       101

    accuracy                           0.69      1934
   macro avg       0.64      0.57      0.60      1934
weighted avg       0.69      0.69      0.69      1934



array([[920, 244,  14,   5],
       [223, 324,   0,   2],
       [ 29,   2,  51,  19],
       [ 37,   1,  20,  43]])